In [1]:
from metadrive.envs.metadrive_env import MetaDriveEnv
from metadrive.policy.expert_policy import ExpertPolicy
from metadrive.examples.ppo_expert.torch_expert import torch_expert as expert

from IPython.display import Image, clear_output
import pandas as pd

from pprint import pprint

## Run scenarios


In [2]:
def run_scenario_n_times_correct(
    n_scenarios=10,
    expert_driving=True,
    use_same_seed=False,
    gif=False,
    user_config: dict = {},
) -> list:
    """
    Runs same scenario n time and collects the traces
    """

    traces = []
    config = user_config | {
        "map": 3,
        "num_scenarios": n_scenarios,
    }

    try:

        for rep in range(n_scenarios):

            env = MetaDriveEnv(config=config)
            seed = 0 if use_same_seed else rep

            obs, step_info = env.reset(seed)
            step_info["repetition"] = rep
            traces.append(step_info)
            print(f"{env.current_seed = }")
            while True:

                # get action from expert driving, or a dummy action
                action = (
                    expert(env.agent, deterministic=True)
                    if expert_driving
                    else [0.1, 0.33]
                )
                obs, reward, tm, tr, step_info = env.step(action)
                step_info["repetition"] = rep
                traces.append(step_info)

                if gif:
                    env.render(
                        mode="topdown",
                        # get the overview of the scene
                        film_size=(1000, 1000),
                        screen_size=(1000, 1000),
                        # set camer to map center
                        camera_position=env.current_map.get_center_point(),
                        # auto determine the number of pixels for 1 meter
                        scaling=None,
                        # do not pop window
                        screen_record=True,
                        window=False,
                    )

                if tm or tr:
                    break

            if gif:
                env.top_down_renderer.generate_gif(
                    gif_name=f"seed_{rep}.gif", duration=30
                )

            env.close()
    finally:
        pass

    return traces

In [3]:
def are_traces_deterministic(traces) -> bool:
    df = pd.DataFrame(traces)

    # grouping by repetition to get a list of traces
    traces = df.groupby("repetition")

    # drop index and repetition ID to compare only step info later
    stripped_traces = [
        trace.reset_index(drop=True).drop("repetition", axis=1) for _, trace in traces
    ]

    # iterate over each trace and check if it is equal to the first one
    are_equal_to_first_trace = [
        trace.equals(stripped_traces[0]) for trace in stripped_traces
    ]

    # if all traces are equal to the first, then all traces are equal hence deterministic
    return all(are_equal_to_first_trace)

## Determinism check


In [4]:
same_seed = True
traces = run_scenario_n_times_correct(
    n_scenarios=10, use_same_seed=same_seed, gif=False
)
clear_output()
if same_seed:
    assert are_traces_deterministic(traces), "Traces must be deterministic!"

df = pd.DataFrame(traces)
# df

,overtake_vehicle_num,velocity,steering,acceleration,step_energy,episode_energy,policy,crash_vehicle,crash_object,crash_building,...,max_step,env_seed,crash,step_reward,cost,repetition,action,raw_action,episode_reward,episode_length
0,0,0.000000,0.000000,0.000000,0.000000,0.000000,EnvInputPolicy,False,False,False,...,False,0,False,0.000000,0,0,NaN,NaN,NaN,NaN
1,0,0.275803,-0.128746,1.000000,0.000217,0.000217,EnvInputPolicy,False,False,False,...,False,0,False,0.007856,0,0,"[-0.12874566, 1.0]","(-0.12874566, 1.0)",0.007856,1.0
2,0,0.551606,-0.120055,1.000000,0.001097,0.001315,EnvInputPolicy,False,False,False,...,False,0,False,0.035550,0,0,"[-0.120055474, 1.0]","(-0.120055474, 1.0)",0.043406,2.0
3,0,0.827386,-0.115118,1.000000,0.002032,0.003346,EnvInputPolicy,False,False,False,...,False,0,False,0.064335,0,0,"[-0.11511787, 1.0]","(-0.11511787, 1.0)",0.107741,3.0
4,0,1.103185,-0.109745,1.000000,0.002984,0.006331,EnvInputPolicy,False,False,False,...,False,0,False,0.093113,0,0,"[-0.10974529, 1.0]","(-0.10974529, 1.0)",0.200854,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5555,0,7.878338,0.001103,0.028080,0.033975,22.951700,EnvInputPolicy,False,False,False,...,False,0,False,0.820788,0,9,"[0.0011034277, 0.028080383]","(0.0011034277, 0.028080383)",511.791986,551.0
5556,0,7.886281,0.012219,0.029728,0.034018,22.985718,EnvInputPolicy,False,False,False,...,False,0,False,0.821724,0,9,"[0.012218889, 0.029728333]","(0.012218889, 0.029728333)",512.613711,552.0
5557,0,7.893533,0.020815,0.027627,0.034060,23.019778,EnvInputPolicy,False,False,False,...,False,0,False,0.822751,0,9,"[0.020814512, 0.02762706]","(0.020814512, 0.02762706)",513.436462,553.0
5558,0,7.876933,0.027106,-0.009952,0.034046,23.053825,EnvInputPolicy,False,False,False,...,False,0,False,0.823098,0,9,"[0.027106341, -0.009952387]","(0.027106341, -0.009952387)",514.259559,554.0


## Fidelity parameters

### Default settings

- `dt = 0.02`
- `decision_repeat = 5`
- Step time is `dt * decision_repeat = 0.1` which is 10FPS


In [5]:
default_fidelity = run_scenario_n_times_correct(
    n_scenarios=1, user_config={"decision_repeat": 5}, gif=True
)

[INFO] Environment: MetaDriveEnv
[INFO] MetaDrive version: 0.4.2.3
[INFO] Sensors: [lidar: Lidar(), side_detector: SideDetector(), lane_line_detector: LaneLineDetector()]
[INFO] Render Mode: none
[INFO] Horizon (Max steps per agent): None
[INFO] Assets version: 0.4.2.3
[INFO] Known Pipes: glxGraphicsPipe
[INFO] Start Scenario Index: 0, Num Scenarios : 1


env.current_seed = 0


[INFO] Episode ended! Scenario Index: 0 Reason: arrive_dest.


In [12]:
low_fidelity = run_scenario_n_times_correct(
    n_scenarios=10, user_config={"decision_repeat": 10}, gif=True
)

[INFO] Environment: MetaDriveEnv
[INFO] MetaDrive version: 0.4.2.3
[INFO] Sensors: [lidar: Lidar(), side_detector: SideDetector(), lane_line_detector: LaneLineDetector()]
[INFO] Render Mode: none
[INFO] Horizon (Max steps per agent): None
[INFO] Assets version: 0.4.2.3
[INFO] Known Pipes: glxGraphicsPipe
[INFO] Start Scenario Index: 0, Num Scenarios : 1


env.current_seed = 0


[INFO] Episode ended! Scenario Index: 0 Reason: arrive_dest.


In [9]:
pd.DataFrame(default_fidelity).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 556 entries, 0 to 555
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   overtake_vehicle_num  556 non-null    int64  
 1   velocity              556 non-null    float64
 2   steering              556 non-null    float64
 3   acceleration          556 non-null    float64
 4   step_energy           556 non-null    float64
 5   episode_energy        556 non-null    float64
 6   policy                556 non-null    object 
 7   crash_vehicle         556 non-null    bool   
 8   crash_object          556 non-null    bool   
 9   crash_building        556 non-null    bool   
 10  crash_human           556 non-null    bool   
 11  crash_sidewalk        556 non-null    bool   
 12  out_of_road           556 non-null    bool   
 13  arrive_dest           556 non-null    bool   
 14  max_step              556 non-null    bool   
 15  env_seed              5